In [1]:
library(lme4)
library(afex)
library(emmeans)
library(dplyr)
library(ggplot2)
library(svglite)
library(lmtest)
library(tidyr)
library(MCMCglmm)
library(reshape)
library(gridExtra)
library(cowplot)
library(svglite)
library(data.table)

Loading required package: Matrix

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- Get and set global package options with: afex_options()
- Set sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************


Attaching package: ‘afex’


The following object is masked from ‘package:lme4’:

    lmer



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘tidyr’


The following objects 

In [2]:
ITI <- read.csv('/home/bagotlab/eshaan.i/PRL/FP_PRL_Nov_2021/CLEAN/Data/PRL_ZScore_ITI.csv')
ITI = subset(ITI,ID!=223)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<-ITI[,c('index','time_idx','time','ID','rewards','lp','LeftLeverProb','Day','rt','PFC','vHIP')]
ITI$sex <- as.factor(ifelse(ITI$ID > 212,"Female", "Male"))
ITI$Day <- as.numeric(as.factor(ITI$Day))

Behavior<- ITI %>% group_by(index)%>%summarize(rewards=mean(rewards),ID=mean(ID),rt=mean(rt),lp=mean(lp), Day=mean(Day), LeftLeverProb=mean(LeftLeverProb))
Behavior$TrialType <- as.factor(ifelse(Behavior$rewards==1, 'Rewards', ifelse(Behavior$lp==0, 'Omission', 'Unrewarded')))
Behavior$sex <- as.factor(ifelse(Behavior$ID > 212,"Female", "Male"))

ITI = subset(ITI, lp > 0)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Reward = lag(rewards,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Previous.Reward = lag(Previous.Reward,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.Choice = lead(lp,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.RT = lead(rt,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Choice = lag(lp,n=240))
ITI$Stay <- ifelse(ITI$lp == ITI$Previous.Choice,1,0)
ITI$StayNext <- ifelse(ITI$lp == ITI$Next.Choice,1,0)

enditi<- as.data.frame(ITI[((ITI$time)>=8)&((ITI$time)<=10),] %>% 
                group_by(index)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),rewards=mean(rewards),ID=mean(ID),Previous.Reward=mean(Previous.Reward), Previous.Previous.Reward=mean(Previous.Previous.Reward),Stay=mean(Stay), StayNext=mean(StayNext), lp=mean(lp),Day=mean(Day)))
Post_DF <- as.data.frame(ITI[((ITI$time)>=0)&((ITI$time)<=2),] %>% group_by(index)%>%summarize(PFC_Post=mean(PFC),vHIP_Post=mean(vHIP)))
Pre_DF <- as.data.frame(ITI[((ITI$time)>=-2)&((ITI$time)<0),] %>% group_by(index)%>%summarize(PFC_Pre=mean(PFC),vHIP_Pre=mean(vHIP)))

DF <- merge(enditi, Post_DF)
DF <- merge(DF, Pre_DF)
DF$sex <- as.factor(ifelse(DF$ID > 212,"Female", "Male"))
DF$ID<-as.factor(DF$ID)
DF$Day<-as.factor(DF$Day)
DF$rewards <- as.factor(ifelse(DF$rewards == 0,"Unrewarded", "Rewarded"))
DF$Previous.Reward <- as.factor(ifelse(DF$Previous.Reward == 0,"Unrewarded", "Rewarded"))
DF$Stay <- as.factor(ifelse(DF$Stay == 0,"Shift", "Stay"))
DF$lp <- as.factor(ifelse(DF$lp == 1,"Contralateral", "Ipsilateral"))

ITI$rewards<-as.factor(ITI$rewards)
ITI$ID<-as.factor(ITI$ID)
ITI$Day<-as.factor(ITI$Day)
ITI$Previous.Reward <-as.factor(ITI$Previous.Reward)
ITI$rewards <- as.factor(ifelse(ITI$rewards == 0,"Unrewarded", "Rewarded"))
ITI$Stay <- as.factor(ifelse(ITI$Stay == 0,"Shift", "Stay"))
ITI$lp <- as.factor(ifelse(ITI$lp == 1,"Contralateral", "Ipsilateral"))


In [3]:
plot_theme <- theme(axis.title=element_text(size=25,color='gray30'),
                    axis.line=element_line(color='gray30'),
                    axis.text.y=element_text(size=20,color='gray30'),
                    axis.text.x=element_text(size=20,color='gray30'),
                    legend.text=element_text(size=20),
                    legend.key.size=unit(2,'mm'),
                    panel.background=element_blank(),
                    panel.grid.major.y = element_blank(),
                    legend.position = 'none',
                    plot.margin = margin(0.5,0.5,0.5,0.5,'cm'),
                    strip.background = element_blank(),
                    strip.text = element_text(size=20,color='gray30')) 
plot_theme <- list(plot_theme, scale_color_manual(values=c('Female'="#8c1aff", 'Male'="#008000",'1'="blue", '0'="red")), 
                    scale_shape_manual(values=c('Male'=16,'Female'=17)),
                    scale_x_continuous(breaks = 1:6))

In [4]:
BulkBehavior <- as.data.frame(Behavior) %>% group_by(ID,sex, Day, TrialType) %>% summarise(Trials=n())
Total_Trials <- subset(Behavior, lp>0) %>% group_by(ID,sex, Day) %>% summarise(Trials=n())
RewardUnreward_Trials <- subset(Behavior, lp>0) %>% group_by(ID,Day,sex,TrialType) %>% summarise(Trials=n())
RewardUnreward_Trials <- RewardUnreward_Trials%>% group_by(ID,sex,Day) %>% mutate(Probability = Trials/sum(Trials))
RewardUnreward_Trials = subset(RewardUnreward_Trials, TrialType=='Rewards')

`summarise()` has grouped output by 'ID', 'sex', 'Day'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'ID', 'sex'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'ID', 'Day', 'sex'. You can override using
the `.groups` argument.


In [ ]:
summary <- BulkBehavior %>% group_by(Day,sex,TrialType) %>% summarise(mean = mean(Trials), sd = sd(Trials),n = n(),se = sd / sqrt(n))
a<-ggplot(subset(BulkBehavior, TrialType=='Unrewarded')) +
  geom_line(aes(x=Day, y=Trials,color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Trials, color=sex, shape=sex ), position = position_jitter(width = .1)) +
  geom_errorbar(data = subset(summary, TrialType=='Unrewarded') , aes(x = Day, ymin = mean-se, ymax=mean+se, color = sex), width=.3)+
  ylim(0,200) +
  plot_theme +
  ylab('Unrewarded Trials')

b<-ggplot(subset(BulkBehavior, TrialType=='Rewards')) +
  geom_line(aes(x=Day, y=Trials,color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Trials, color=sex, shape=sex ), position = position_jitter(width = .1)) +
  geom_errorbar(data = subset(summary, TrialType=='Rewards') , aes(x = Day, ymin = mean-se, ymax=mean+se, color = sex), width=.3)+
  ylim(0,200) +
  plot_theme +
  ylab('Rewarded Trials')

c<-ggplot(subset(BulkBehavior, TrialType=='Omission')) +
  geom_line(aes(x=Day, y=Trials,color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Trials, color=sex, shape=sex ), position = position_jitter(width = .1)) +
  geom_errorbar(data = subset(summary, TrialType=='Omission') , aes(x = Day, ymin = mean-se, ymax=mean+se, color = sex), width=.3)+
  ylim(0,200) +
  plot_theme +
  ylab('Omitted Trials')

#figure C
omissionlmer<- lmer(Trials~sex*Day+(1+Day|ID), data=subset(BulkBehavior, TrialType=='Omission'))
anova(omissionlmer)
#figure B
rewardslmer<- lmer(Trials~sex*Day+(1+Day|ID), data=subset(BulkBehavior, TrialType=='Rewards'))
anova(rewardslmer)
#figure A
unrewardedlmer<- lmer(Trials~sex*Day+(1+Day|ID), data=subset(BulkBehavior, TrialType=='Unrewarded'))
anova(unrewardedlmer)

`summarise()` has grouped output by 'Day', 'sex'. You can override using the
`.groups` argument.


boundary (singular) fit: see help('isSingular')



,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,444.2498,444.2498,1,19.79239,2.661546,1.186109e-01
Day,8600.7041,8600.7041,1,20.85532,51.527697,4.658086e-07
sex:Day,362.7994,362.7994,1,20.85532,2.173568,1.553362e-01


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,620.8255,620.8255,1,19.86365,2.687610,1.168736e-01
Day,32044.5691,32044.5691,1,19.93285,138.723860,1.969770e-10
sex:Day,710.5397,710.5397,1,19.93285,3.075991,9.482018e-02


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,360.8958,360.8958,1,19.77454,1.116991,0.3033086
Day,1289.7992,1289.7992,1,19.93138,3.991993,0.0595468
sex:Day,326.1511,326.1511,1,19.93138,1.009454,0.3270763


In [ ]:
summary <- Total_Trials %>% group_by(Day, sex) %>% summarise(mean = mean(Trials), sd = sd(Trials),n = n(),se = sd / sqrt(n))
d<-ggplot(Total_Trials) +
  geom_line(aes(x=Day, y=Trials, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Trials, color=sex, shape=sex ), position = position_jitter(width = .1)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylim(0,335) +
  ylab('Trials Completed') +
  plot_theme

#figure G
trialslmer<- lmer(Trials~sex*Day+(1|ID), data=Total_Trials)
anova(trialslmer) 

`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,4693.491,4693.491,1,61.34283,6.074702,1.652850e-02
Day,116197.575,116197.575,1,104.75959,150.392470,5.655848e-22
sex:Day,4634.382,4634.382,1,104.75959,5.998199,1.597955e-02


In [ ]:
summary <- RewardUnreward_Trials %>% group_by(Day, sex) %>% summarise(mean = mean(Probability), sd = sd(Probability),n = n(),se = sd / sqrt(n))
e<-ggplot(RewardUnreward_Trials) +
  geom_line(aes(x=Day, y=Probability, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Probability, color=sex,shape=sex ), position = position_jitter(width = .2)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylab('P(Reward)') +
  plot_theme

#figure D
trialslmer<- lmer(Probability~sex*Day+(1+Day|ID), data=RewardUnreward_Trials)
anova(trialslmer) 

`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,0.001520175,0.001520175,1,19.88936,0.5981883,4.483656e-01
Day,0.152165265,0.152165265,1,19.92607,59.8769646,1.991040e-07
sex:Day,0.002253153,0.002253153,1,19.92607,0.8866146,3.576729e-01


In [14]:
Stay <- subset(Behavior, lp>0) %>% group_by(ID,Day) %>% mutate(NextChoice = lead(lp), Stay = ifelse(NextChoice==lp, 1,0))
winstay<-na.omit(subset(Stay, rewards==1))%>% group_by(ID,Day,sex,Stay) %>% summarise(Trials=n())
winstay<- winstay%>% group_by(ID,sex,Day) %>% mutate(Probability = Trials/sum(Trials))
winstay<- subset(winstay, Stay==1)

losestay<-na.omit(subset(Stay, rewards==0))%>% group_by(ID,Day,sex,Stay) %>% summarise(Trials=n())
losestay<- losestay%>% group_by(ID,sex,Day) %>% mutate(Probability = Trials/sum(Trials))
losestay<- subset(losestay, Stay==1)

`summarise()` has grouped output by 'ID', 'Day', 'sex'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'ID', 'Day', 'sex'. You can override using
the `.groups` argument.


In [ ]:
summary <- winstay %>% group_by(Day, sex) %>% summarise(mean = mean(Probability), sd = sd(Probability),n = n(),se = sd / sqrt(n))
f<-ggplot(winstay) +
  geom_line(aes(x=Day, y=Probability, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Probability, color=sex,shape=sex ), position = position_jitter(width = .2)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylab('P(Stay|Reward)') +
  plot_theme

#figure E
trialslmer<- lmer(Probability~sex*Day+(1+Day|ID), data=winstay)
anova(trialslmer) 

`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.
boundary (singular) fit: see help('isSingular')



,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,1.977240e-02,1.977240e-02,1,20.11918,2.907095784,0.1035846
Day,9.020176e-06,9.020176e-06,1,21.76807,0.001326218,0.9712815
sex:Day,1.336212e-02,1.336212e-02,1,21.76807,1.964606345,0.1751195


In [ ]:
summary <- losestay %>% group_by(Day, sex) %>% summarise(mean = mean(Probability), sd = sd(Probability),n = n(),se = sd / sqrt(n))
g<-ggplot(losestay) +
  geom_line(aes(x=Day, y=Probability, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Probability, color=sex,shape=sex ), position = position_jitter(width = .2)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylab('P(Stay|Unrewarded)') +
  plot_theme

#figure F
trialslmer<- lmer(Probability~sex*Day+(1+Day|ID), data=losestay)
anova(trialslmer) 

`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,0.005364235,0.005364235,1,19.93266,0.7783582,3.881596e-01
Day,0.345077360,0.345077360,1,19.91325,50.0712210,7.535722e-07
sex:Day,0.016803173,0.016803173,1,19.91325,2.4381645,1.341673e-01


In [28]:
lp_reversals<-subset(Behavior, lp>0) %>% group_by(run = data.table::rleid(ID,Day, sex,LeftLeverProb), ID,Day,sex) %>% summarise(count = n())
reversals_completed <- lp_reversals %>% group_by(ID, Day,sex) %>% summarise(Reversals=n()-1)
lp_reversals <- merge(lp_reversals,reversals_completed)

`summarise()` has grouped output by 'run', 'ID', 'Day'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'ID', 'Day'. You can override using the
`.groups` argument.


In [ ]:
summary <- reversals_completed %>% group_by(Day, sex) %>% summarise(mean = mean(Reversals), sd = sd(Reversals),n = n(),se = sd / sqrt(n))
h<-ggplot(reversals_completed) +
  geom_line(aes(x=Day, y=Reversals, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = Reversals, color=sex,shape=sex ), position = position_jitter(width = .2)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylab('Reversals Completed') +
  plot_theme 

#figure H
reversalslmer<- lmer(Reversals~sex*Day+(1+Day|ID), data=reversals_completed)
anova(reversalslmer) 

`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,0.6798587,0.6798587,1,19.71955,0.1626645,6.910547e-01
Day,168.8463298,168.8463298,1,19.99131,40.3985564,3.349112e-06
sex:Day,1.2631811,1.2631811,1,19.99131,0.3022316,5.885766e-01


In [ ]:
Avg_LP_Block <- lp_reversals %>% group_by(ID,Day,sex) %>% summarise(AverageLP=mean(count))
summary <- Avg_LP_Block %>% group_by(Day, sex) %>% summarise(mean = mean(AverageLP), sd = sd(AverageLP),n = n(),se = sd / sqrt(n))
i<-ggplot(Avg_LP_Block) +
  geom_line(aes(x=Day, y=AverageLP, color=sex), stat="summary", fun='mean', alpha=0.6, size=2) +
  geom_jitter(aes(x = Day, y = AverageLP, color=sex,shape=sex ), position = position_jitter(width = .2)) +
  geom_errorbar(data = summary, aes(x = Day, ymin = mean-se, ymax=mean+se, color=sex), width=.3)+
  ylab('Lever Presses per Block') +
  plot_theme 

#figure I
reversalslmer<- lmer(AverageLP~sex*Day+(1+Day|ID), data=Avg_LP_Block)
anova(reversalslmer) 

`summarise()` has grouped output by 'ID', 'Day'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'Day'. You can override using the `.groups`
argument.


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
sex,271.1667,271.1667,1,20.02493,0.1751981,0.6799878231
Day,25655.9040,25655.9040,1,20.01392,16.5760290,0.0005946743
sex:Day,384.4756,384.4756,1,20.01392,0.2484059,0.6236328078
